### Résultats de l'assimilation

In [15]:
import psycopg2
import pandas as pd
import re
import os

# Chemin vers le dossier principal contenant les sous-dossiers (villes)
dossier_principal = "dossier_code"
dossier_destination = 'Fiche'

# Dictionnaire pour stocker les informations par ville, y compris les listes
infos_villes = {}

# Parcours des dossiers (villes)
for ville in os.listdir(dossier_principal):
    chemin_ville = os.path.join(dossier_principal, ville)

    # Vérification si l'élément est un dossier
    if os.path.isdir(chemin_ville):
        # Initialisation des listes de codes postaux uniques pour cette ville
        codes_postaux = {
            "15_min": set(),
            "30_min": set(),
            "45_min": set(),
            "total": set()  # Ajout de la liste totale
        }

        for fichier in os.listdir(chemin_ville):
            chemin_fichier = os.path.join(chemin_ville, fichier)

            # Vérification si l'élément est un fichier Excel
            if os.path.isfile(chemin_fichier) and chemin_fichier.endswith(".xlsx"):
                # Lecture du fichier Excel
                df = pd.read_excel(chemin_fichier)

                codes = set(df["Code postal"].tolist())  # Convertir en ensemble (set)

                # Mise à jour des codes postaux uniques par minute
                if fichier == "15.xlsx":
                    codes_postaux["15_min"].update(codes)
                elif fichier == "30.xlsx":
                    codes_postaux["30_min"].update(codes - codes_postaux["15_min"])
                elif fichier == "45.xlsx":
                    codes_postaux["45_min"].update(codes - codes_postaux["30_min"] - codes_postaux["15_min"])

                # Mise à jour de la liste totale
                codes_postaux["total"].update(codes)

        # Stockage des informations par ville, y compris les listes
        nom_dossier = os.path.basename(dossier_principal)
        nom_ville = os.path.basename(chemin_ville)
        infos_villes[nom_ville] = {
            "Nom_Dossier": nom_dossier,
            "Nom_Ville": nom_ville,
            "Codes_Postaux_15_min": len(codes_postaux["15_min"]),
            "Codes_Postaux_30_min": len(codes_postaux["30_min"]),
            "Codes_Postaux_45_min": len(codes_postaux["45_min"]),
            "Codes_Postaux_Uniques": len(codes_postaux["total"]),  # Utilisation de la liste totale pour les codes uniques
            "Codes_Postaux_Listes": codes_postaux  # Ajout des listes de codes postaux
        }

# Affichage des informations par ville et des listes de codes postaux uniques
for nom_ville, info in infos_villes.items():
    print(f"Dossier principal: {info['Nom_Dossier']}")
    print(f"Nom de la ville: {info['Nom_Ville']}")
    print("Nombre de codes postaux uniques de 15 min:", info["Codes_Postaux_15_min"])
    print("Nombre de codes postaux uniques de 30 min:", info["Codes_Postaux_30_min"])
    print("Nombre de codes postaux uniques de 45 min:", info["Codes_Postaux_45_min"])
    print("Nombre de codes postaux uniques au total:", info["Codes_Postaux_Uniques"])
    print("Liste de codes postaux uniques de 15 min:", info["Codes_Postaux_Listes"]["15_min"])
    print("Liste de codes postaux uniques de 30 min:", info["Codes_Postaux_Listes"]["30_min"])
    print("Liste de codes postaux uniques de 45 min:", info["Codes_Postaux_Listes"]["45_min"])
    print("Liste de codes postaux uniques au total:", info["Codes_Postaux_Listes"]["total"])
    print("\n")

# Établir la connexion à la base de données PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="Base",
    user="waouhmonde",
    password="waouhmonde"
)

for nom_ville, info in infos_villes.items():
    print(f"Traitement de la ville : {nom_ville}")
    for valeur_type in ["15_min", "30_min", "45_min", "total"]:
        print(f"{nom_ville} - {valeur_type}")
        codes_postaux_liste = info["Codes_Postaux_Listes"][valeur_type]
        # Convertissez les codes postaux en chaînes de caractères (str)
        codes_postaux_str = [str(code_postal) for code_postal in codes_postaux_liste]

        # Créez une requête SQL pour rechercher les lignes correspondantes dans votre table
        query = "SELECT * FROM cho WHERE cp IN %s"

        # Exécutez la requête SQL en utilisant les codes postaux convertis comme paramètres
        df = pd.read_sql_query(query, conn, params=(tuple(codes_postaux_str),))
        print(df)

        # Si le DataFrame est vide, affichez un message
        if df.empty:
            print("Aucun résultat trouvé pour cette ville.")
        else:
            # Supprimer les doublons en fonction du numéro de téléphone
            df.drop_duplicates(subset=['tel'], keep='first', inplace=True)

            def extract_nom_prenom(row):
                nom_prenom = row['npn'].split(' ')
                nom = nom_prenom[0]
                prenom = ' '.join(nom_prenom[1:])
                return pd.Series([nom, prenom], index=['Nom', 'Prenom'])

            # Appliquer la fonction au DataFrame et concaténer le résultat
            df = pd.concat([df, df.apply(extract_nom_prenom, axis=1)], axis=1)

            # Supprimer la colonne 'nom' originale si nécessaire
            df = df.drop('npn', axis=1)

            # Lire le fichier Excel dans df1
            nom_du_fichier = 'prenoms_arabes.xlsx'
            df1 = pd.read_excel(nom_du_fichier)

            prenoms_arabes = df1['Prénoms']

            # Filtrer les prénoms arabes
            masque = df['Prenom'].isin(prenoms_arabes)
            df = df[~masque]
            df.rename(columns={'cp': 'Code postal'}, inplace=True)
            df.rename(columns={'dn': 'Email'}, inplace=True)
            df = df[['Nom', 'Prenom'] + [col for col in df.columns if col not in ['Nom', 'Prenom']]]

            # Définition des fonctions pour les numéros mobiles et fixes
            def est_mobile(numero):
                motif_mobile = r'^(6\d{8}|\+336\d{8}|7\d{8})'  # Inclure les numéros commençant par "7" comme numéros mobiles
                return bool(re.match(motif_mobile, str(numero)))  # Convertir en chaîne de caractères

            def est_fixe(numero):
                motif_fixe = r'^(0[1-5]\d{7}|0[8-9]\d{7}|[1-5]\d{7}|[6-9]\d{7}|9\d{7})'  # Inclure les numéros commençant par "9" comme numéros fixes
                return bool(re.match(motif_fixe, str(numero)))  # Convertir en chaîne de caractères

            # Filtrer les numéros mobiles et fixes
            df_mobile = df[df['tel'].apply(est_mobile)].copy()
            df_mobile.rename(columns={'tel': 'N° Mobile'}, inplace=True)
            
            df_fixe = df[df['tel'].apply(est_fixe)].copy()
            df_fixe.rename(columns={'tel': 'N° Fixe'}, inplace=True)
            df_inconnu = df[~df['tel'].apply(est_mobile) & ~df['tel'].apply(est_fixe)].copy()
            

            # Obtenir une liste des numéros inconnus
            numeros_inconnus = df_inconnu['tel'].tolist()

            # Compter les lignes où le numéro est vide ou None
            numero_vide_count = df[df['tel'].isna() | df['tel'] == ''].shape[0]

            # Nombre de lignes pour les numéros fixes
            nombre_fixe = df_fixe.shape[0]

            # Nombre de lignes pour les numéros mobiles
            nombre_mobile = df_mobile.shape[0]

            # Somme des numéros fixes et mobiles
            somme_fixe_mobile = nombre_fixe + nombre_mobile

            print(f"Nombre de lignes avec des numéros vides ou None : {numero_vide_count}")
            print(f"Nombre de lignes pour les numéros fixes : {nombre_fixe}")
            print(f"Nombre de lignes pour les numéros mobiles : {nombre_mobile}")
            print(f"Somme des numéros fixes et mobiles : {somme_fixe_mobile}")

            print("\n")
            
            # Assurez-vous que le dossier de destination existe, sinon, créez-le
            if not os.path.exists(dossier_destination):
                os.makedirs(dossier_destination)

            # Initialisez un compteur de partie
            partie = 1
            chunk_size = 100000  # Nombre de lignes dans chaque morceau

            # Tant qu'il y a plus de lignes restantes dans le DataFrame
            while not df_fixe.empty:
                # Sélectionnez les premières lignes (un morceau)
                df_partie = df_fixe.head(chunk_size)
    
                # Enregistrez la partie dans un fichier Excel dans le dossier de destination
                nom_fichier_partie = os.path.join(dossier_destination, f"{nom_ville}-{valeur_type}Fixe-partie{partie}.xlsx")
                df_partie.to_excel(nom_fichier_partie, index=False)

                # Supprimez les lignes extraites du DataFrame d'origine
                df_fixe = df_fixe.iloc[chunk_size:]

                # Incrémentez le compteur de partie
                partie += 1

            print(f"Fichiers créés avec succès dans le dossier de destination : {dossier_destination}")
   
conn.close()

Dossier principal: dossier_code
Nom de la ville: PLOUAY
Nombre de codes postaux uniques de 15 min: 6
Nombre de codes postaux uniques de 30 min: 24
Nombre de codes postaux uniques de 45 min: 23
Nombre de codes postaux uniques au total: 53
Liste de codes postaux uniques de 15 min: {56320, 56240, 56850, 29300, 56650, 56620}
Liste de codes postaux uniques de 30 min: {56260, 29380, 56520, 56270, 56530, 56600, 29340, 56540, 56670, 56160, 56290, 56100, 29350, 56680, 56110, 29360, 56560, 56690, 56630, 56310, 56440, 56570, 56700, 29310}
Liste de codes postaux uniques de 45 min: {56770, 22340, 56330, 29900, 29390, 56400, 29520, 56340, 29140, 56150, 29910, 29270, 56410, 22110, 56480, 29920, 56930, 29540, 56550, 29930, 56300, 56500, 56950}
Liste de codes postaux uniques au total: {56320, 56330, 56850, 56340, 56600, 56100, 56620, 56110, 56630, 22340, 56650, 56400, 29520, 56150, 29270, 56410, 56670, 22110, 56160, 56930, 29540, 56680, 56690, 29300, 56950, 56440, 56700, 29310, 29340, 56480, 29350, 562

C:\Users\Admin\AppData\Local\Temp\ipykernel_4204\2138958917.py:95: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(tuple(codes_postaux_str),))


                    npn    dn                      adresse     cp  \
0        Le Gal Romuald  None         2 penhoet saint lalu  56240   
1         tanguy emilie  None               fontaine maria  56240   
2       guily gwenaelle  None                 coet neblech  56240   
3       nicolas monique  None                 12 kermarrec  56240   
4          LAYEC EMILIE  None                  56240 BERNE  56240   
...                 ...   ...                          ...    ...   
2223   daniel catherine  None                3 rue stankou  56620   
2224        dias sylvie  None          19 rue anjela duval  56620   
2225  le romancer nadia  None             lieu dit biviere  56620   
2226      RAOUL MONIQUE  None            56620 PONT SCORFF  56620   
2227    le roux donovan  None  14 rue du general de gaulle  56620   

            ville        tel email  
0       inguiniel  647184130  None  
1           calan  688083488  None  
2          plouay  687402341  None  
3          plouay  6577

C:\Users\Admin\AppData\Local\Temp\ipykernel_4204\2138958917.py:95: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(tuple(codes_postaux_str),))


                           npn    dn                            adresse  \
0                  MAHUAS JEAN  None                      56100 LORIENT   
1              le hel severine  None               25 rue de kervenanec   
2            Le bihan Lauriane  None  102 rue general de la bollardiere   
3                 watin noemie  None      1 rue paul vaillant couturier   
4             BELVERGE MICKAEL  None                      56100 LORIENT   
...                        ...   ...                                ...   
11696   TOULLIOU ENVIRONNEMENT  None                      SAINT-LAURENT   
11697            bourglan gael  None                          11 le cap   
11698            PENNEC NOALIG  None                       56520 GUIDEL   
11699        bernery eloÃÂ¯se  None       29 rue de la villeneuve elle   
11700        bouyaux gabrielle  None                            kerhars   

          cp    ville        tel email  
0      56100  LORIENT  688341789  None  
1      56100  lor

C:\Users\Admin\AppData\Local\Temp\ipykernel_4204\2138958917.py:95: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(tuple(codes_postaux_str),))


                      npn                    dn  \
0           POUZET MORGAN                  None   
1       pierache nathalie  nathpiera@hotmail.fr   
2        Jeanvoine Amélie                  None   
3          Julien Bernard                  None   
4     le CLEC'H GaÃÂ«tan                  None   
...                   ...                   ...   
8042        Stephane Fath                  None   
8043        evano aurelie                  None   
8044      guegan angeline                  None   
8045    vasseur christine                  None   
8046        floch martine                  None   

                                 adresse     cp              ville        tel  \
0                  7 LIEU DIT KERMANAC H  22110  PLOUNEVEZ QUINTIN  754003770   
1                      12 rue du hambout  22110          rostrenen  615777354   
2                             22110 NULL  22110               None  679257855   
3                   2 Rue De La Fontaine  22110          Rostre

C:\Users\Admin\AppData\Local\Temp\ipykernel_4204\2138958917.py:95: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(tuple(codes_postaux_str),))


                       npn                     dn                adresse  \
0        pierache nathalie   nathpiera@hotmail.fr      12 rue du hambout   
1      mauffray jacqueline  mauffraybj@hotmail.fr       rue de koadernod   
2           Nicole Terriot                   None    1 imp de pen ar roz   
3         KERNOA CATHERINE                   None       22110 MELLIONNEC   
4          MAURIEGE SANDRA                   None                 22110    
...                    ...                    ...                    ...   
21971    Sylvie HILLAIREAU                   None              Kersolard   
21972    bourbon dominique                   None  35 lieu dit kerdolmen   
21973        lydie Doudard                   None         5 Rue du Stade   
21974      jegousse emilie                   None     5 lieu dit kernaud   
21975   Adrian Le Chenadec                   None          2 La Fontaine   

          cp       ville        tel email  
0      22110   rostrenen  615777354  None  

In [ ]:
import psycopg2
import pandas as pd
import re
import os

# Chemin vers le dossier principal contenant les sous-dossiers (villes)
dossier_principal = "dossier_code"

# Dictionnaire pour stocker les informations par ville, y compris les listes
infos_villes = {}

# Parcours des dossiers (villes)
for ville in os.listdir(dossier_principal):
    chemin_ville = os.path.join(dossier_principal, ville)

    # Vérification si l'élément est un dossier
    if os.path.isdir(chemin_ville):
        # Initialisation des listes de codes postaux uniques pour cette ville
        codes_postaux = {
            "15_min": set(),
            "30_min": set(),
            "45_min": set(),
            "total": set()  # Ajout de la liste totale
        }

        for fichier in os.listdir(chemin_ville):
            chemin_fichier = os.path.join(chemin_ville, fichier)

            # Vérification si l'élément est un fichier Excel
            if os.path.isfile(chemin_fichier) and chemin_fichier.endswith(".xlsx"):
                # Lecture du fichier Excel
                df = pd.read_excel(chemin_fichier)

                codes = set(df["Code postal"].tolist())  # Convertir en ensemble (set)

                # Mise à jour des codes postaux uniques par minute
                if fichier == "15.xlsx":
                    codes_postaux["15_min"].update(codes)
                elif fichier == "30.xlsx":
                    codes_postaux["30_min"].update(codes - codes_postaux["15_min"])
                elif fichier == "45.xlsx":
                    codes_postaux["45_min"].update(codes - codes_postaux["30_min"] - codes_postaux["15_min"])

                # Mise à jour de la liste totale
                codes_postaux["total"].update(codes)

        # Stockage des informations par ville, y compris les listes
        nom_dossier = os.path.basename(dossier_principal)
        nom_ville = os.path.basename(chemin_ville)
        infos_villes[nom_ville] = {
            "Nom_Dossier": nom_dossier,
            "Nom_Ville": nom_ville,
            "Codes_Postaux_15_min": len(codes_postaux["15_min"]),
            "Codes_Postaux_30_min": len(codes_postaux["30_min"]),
            "Codes_Postaux_45_min": len(codes_postaux["45_min"]),
            "Codes_Postaux_Uniques": len(codes_postaux["total"]),  # Utilisation de la liste totale pour les codes uniques
            "Codes_Postaux_Listes": codes_postaux  # Ajout des listes de codes postaux
        }

# Affichage des informations par ville et des listes de codes postaux uniques
for nom_ville, info in infos_villes.items():
    print(f"Dossier principal: {info['Nom_Dossier']}")
    print(f"Nom de la ville: {info['Nom_Ville']}")
    print("Nombre de codes postaux uniques de 15 min:", info["Codes_Postaux_15_min"])
    print("Nombre de codes postaux uniques de 30 min:", info["Codes_Postaux_30_min"])
    print("Nombre de codes postaux uniques de 45 min:", info["Codes_Postaux_45_min"])
    print("Nombre de codes postaux uniques au total:", info["Codes_Postaux_Uniques"])
    print("Liste de codes postaux uniques de 15 min:", info["Codes_Postaux_Listes"]["15_min"])
    print("Liste de codes postaux uniques de 30 min:", info["Codes_Postaux_Listes"]["30_min"])
    print("Liste de codes postaux uniques de 45 min:", info["Codes_Postaux_Listes"]["45_min"])
    print("Liste de codes postaux uniques au total:", info["Codes_Postaux_Listes"]["total"])
    print("\n")

# Établir la connexion à la base de données PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="Base",
    user="waouhmonde",
    password="waouhmonde"
)

for nom_ville, info in infos_villes.items():
    print(f"Traitement de la ville : {nom_ville}")
    for valeur_type in ["15_min", "30_min", "45_min", "total"]:
        print(f"{nom_ville} - {valeur_type}")
        codes_postaux_liste = info["Codes_Postaux_Listes"][valeur_type]
        # Convertissez les codes postaux en chaînes de caractères (str)
        codes_postaux_str = [str(code_postal) for code_postal in codes_postaux_liste]

        # Créez une requête SQL pour rechercher les lignes correspondantes dans votre table
        query = "SELECT * FROM ma_table WHERE code_postal IN %s"

        # Exécutez la requête SQL en utilisant les codes postaux convertis comme paramètres
        df = pd.read_sql_query(query, conn, params=(tuple(codes_postaux_str),))

        # Si le DataFrame est vide, affichez un message
        if df.empty:
            print("Aucun résultat trouvé pour cette ville.")
        else:
            # Supprimer les doublons en fonction du numéro de téléphone
            df.drop_duplicates(subset=['numero_telephone'], keep='first', inplace=True)

            def extract_nom_prenom(row):
                nom_prenom = row['nom'].split(' ')
                nom = nom_prenom[0]
                prenom = ' '.join(nom_prenom[1:])
                return pd.Series([nom, prenom], index=['Nom', 'Prenom'])

            # Appliquer la fonction au DataFrame et concaténer le résultat
            df = pd.concat([df, df.apply(extract_nom_prenom, axis=1)], axis=1)

            # Supprimer la colonne 'nom' originale si nécessaire
            df = df.drop('nom', axis=1)

            # Lire le fichier Excel dans df1
            nom_du_fichier = 'prenoms_arabes.xlsx'
            df1 = pd.read_excel(nom_du_fichier)

            prenoms_arabes = df1['Prénoms']

            # Filtrer les prénoms arabes
            masque = df['Prenom'].isin(prenoms_arabes)
            df = df[~masque]

            # Définition des fonctions pour les numéros mobiles et fixes
            def est_mobile(numero):
                motif_mobile = r'^(6\d{8}|\+336\d{8}|7\d{8})'  # Inclure les numéros commençant par "7" comme numéros mobiles
                return bool(re.match(motif_mobile, str(numero)))  # Convertir en chaîne de caractères

            def est_fixe(numero):
                motif_fixe = r'^(0[1-5]\d{7}|0[8-9]\d{7}|[1-5]\d{7}|[6-9]\d{7}|9\d{7})'  # Inclure les numéros commençant par "9" comme numéros fixes
                return bool(re.match(motif_fixe, str(numero)))  # Convertir en chaîne de caractères

            # Filtrer les numéros mobiles et fixes
            df_mobile = df[df['numero_telephone'].apply(est_mobile)].copy()
            df_fixe = df[df['numero_telephone'].apply(est_fixe)].copy()
            
            if valeur_type == "total" :
                nom_fichier_fixe = f"{nom_ville}Fixe.xlsx"
                nom_fichier_mobile = f"{nom_ville}Mobile.xlsx"
                df_fixe.to_excel(nom_fichier_fixe, index=False)
                df_mobile.to_excel(nom_fichier_mobile, index=False)
                print("fichier chargés")
            # Créer une DataFrame pour les numéros inconnus
            df_inconnu = df[~df['numero_telephone'].apply(est_mobile) & ~df['numero_telephone'].apply(est_fixe)].copy()

            # Obtenir une liste des numéros inconnus
            numeros_inconnus = df_inconnu['numero_telephone'].tolist()

            # Compter les lignes où le numéro est vide ou None
            numero_vide_count = df[df['numero_telephone'].isna() | df['numero_telephone'] == ''].shape[0]

            # Nombre de lignes pour les numéros fixes
            nombre_fixe = df_fixe.shape[0]

            # Nombre de lignes pour les numéros mobiles
            nombre_mobile = df_mobile.shape[0]

            # Somme des numéros fixes et mobiles
            somme_fixe_mobile = nombre_fixe + nombre_mobile

            print(f"Nombre de lignes avec des numéros vides ou None : {numero_vide_count}")
            print(f"Nombre de lignes pour les numéros fixes : {nombre_fixe}")
            print(f"Nombre de lignes pour les numéros mobiles : {nombre_mobile}")
            print(f"Somme des numéros fixes et mobiles : {somme_fixe_mobile}")

            print("\n")

# Fermez la connexion à la base de données
conn.close()